In [116]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Union

In [117]:
M = 1.989e30 #kg
G = 6.6741e-11 # m^3/kg/s^2
AU = 149597870700 #m

In [118]:
x0 = np.array([[0], [0.586]]) * AU # m
v0 = np.array([[54600], [0]]) # m/s


In [119]:
def V(r: np.ndarray) -> np.ndarray:
    """
    Gravitonal Potential Energy dependent on position r = [x, y]
    """
    return -G * M * r  / (np.power(r.dot(r), 3/2))# J

In [120]:
def derivative(func, x0: float, n: int = 1) -> float:
    """
    Computes derivative of a function f where x=x0, using recurrency
    """
    dx = 1e-6
    if n == 1:
        dy = func(x0+dx) - func(x0)
        return dy/dx
    if n > 1:
        dy = derivative(func, x0 + dx, n-1) - derivative(func, x0, n-1)
        return dy / dx

In [121]:
class Solver:
    def __init__(self, tmax: int, dt: float):
        self.x = None
        self.v = None
        self.t = None
        self.dt = None
        self.update_params(tmax, dt)

    def update_params(self, tmax: int, dt: float):
        num = int(tmax//dt)
        self.x = np.zeros((2, num))
        self.v = np.zeros((2, num))
        self.t = np.linspace(0, tmax, num=num)
        self.dt = dt

    def set_new_values(self, x: np.ndarray, v: np.ndarray, i):
        self.x[:][i] = x
        self.v[:][i] = v

    def get_new_values(self, i: int):
        return self.x[:][i], self.v[:][i]

    def calculate(self):
        for i in range(len(self.t)):
            if i == 0:
                self.set_new_values(x0, v0, 0)
                continue
            x, y = self.get_next_step(i-1)
            self.set_new_values(x, y, i)

In [122]:
class Euler(Solver):
    def __init__(self, tmax: int, dt: float):
        super().__init__(tmax, dt)

    def get_next_step(self, i: int):
        last_x = self.x[:][i]
        last_v = self.v[:][i]
        new_x = last_x + last_v * self.dt
        new_y = last_v - derivative(V, last_x) * self.dt
        return new_x, new_y

In [123]:
class Plotter:
    def __init__(self, solver: Euler, dir: str):
        self.solver = solver
        self.dir = dir

    def get_KE(self) -> np.ndarray:
        return 0.5 * M * self.solver.v ** 2

    def get_V(self):
        return V(self.solver.x)

    def get_TE(self):
        return self.get_KE() + self.get_V()

    def plot(self, filename: str = "funcT"):
        plt.plot(self.solver.x[0], self.solver.x[1])
        plt.show()



In [124]:
solv = Euler(10000, 0.1)
solv.calculate()
mplt = Plotter(solv, "")
mplt.plot()

ValueError: could not broadcast input array from shape (2,1) into shape (99999,)

In [125]:
solv.v[0]

array([0., 0., 0., ..., 0., 0., 0.])